In [128]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import seaborn as sb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import tree, ensemble, neighbors, model_selection, metrics

In [129]:
sb.set()

sp = spotipy.Spotify()
        
cid = "6d49981d346842a1844ab1612afae8ba"
csec = "e33407ff688c4d48908d5f5286fe8e41"


client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=csec)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

playlist_tracks = sp.user_playlist_tracks('22b7g6udnfkubfplnsla3ww5a', '7pF07PQEdD4e0AEoo6ue9G', fields='items,uri,name,id,total', market='fr')

name = []
date = []
pop = []
dance = []
energy = []
loud = []
speech = []
acous = []
live = []
valence = []
tempo = []
win = []

for i in playlist_tracks['items']:
    name.append(i['track']['name'])
    date.append(i['track']['album']['release_date'])
    pop.append(i['track']['popularity'])
    dance.append(sp.audio_features('spotify:track:'+i['track']['id'])[0]['danceability'])
    energy.append(sp.audio_features('spotify:track:'+i['track']['id'])[0]['energy'])
    loud.append(sp.audio_features('spotify:track:'+i['track']['id'])[0]['loudness'])
    speech.append(sp.audio_features('spotify:track:'+i['track']['id'])[0]['speechiness'])
    acous.append(sp.audio_features('spotify:track:'+i['track']['id'])[0]['acousticness'])
    live.append(sp.audio_features('spotify:track:'+i['track']['id'])[0]['liveness'])
    valence.append(sp.audio_features('spotify:track:'+i['track']['id'])[0]['valence'])
    tempo.append(sp.audio_features('spotify:track:'+i['track']['id'])[0]['tempo'])
    win.append(1)
    
    
playlist_tracks = sp.user_playlist_tracks('22b7g6udnfkubfplnsla3ww5a', '4OzkYLxp0VzMNlYBXq2faU', fields='items,uri,name,id,total', market='fr')
for i in playlist_tracks['items']:
    name.append(i['track']['name'])
    date.append(i['track']['album']['release_date'])
    pop.append(i['track']['popularity'])
    dance.append(sp.audio_features('spotify:track:'+i['track']['id'])[0]['danceability'])
    energy.append(sp.audio_features('spotify:track:'+i['track']['id'])[0]['energy'])
    loud.append(sp.audio_features('spotify:track:'+i['track']['id'])[0]['loudness'])
    speech.append(sp.audio_features('spotify:track:'+i['track']['id'])[0]['speechiness'])
    acous.append(sp.audio_features('spotify:track:'+i['track']['id'])[0]['acousticness'])
    live.append(sp.audio_features('spotify:track:'+i['track']['id'])[0]['liveness'])
    valence.append(sp.audio_features('spotify:track:'+i['track']['id'])[0]['valence'])
    tempo.append(sp.audio_features('spotify:track:'+i['track']['id'])[0]['tempo'])
    win.append(0)

data = pd.DataFrame()
data.insert(0, "Name", name, True)
data.insert(1, "Date Released", date, True)
data.insert(2, "Popularity", pop, True)
data.insert(3, "Danceability", dance, True)
data.insert(4, "Energy", energy, True)
data.insert(5, "Loudness", loud, True)
data.insert(6, "Speechiness", speech, True)
data.insert(7, "Acousticness", acous, True)
data.insert(8, "Liveness", live, True)
data.insert(9, "Valence", valence, True)
data.insert(10, "Tempo", tempo, True)
data.insert(11, "Did They Win?", win, True)


data

# sp.trace = True # turn on tracing
# sp.trace_out = True # turn on trace out

# urn = 'spotify:artist:3jOstUTkEu2JkjvRdBA5Gu'
# # artist = sp.artist(urn)
# # print(artist.get('popularity'))

# track_uri = 'spotify:track:35xSkNIXi504fcEwz9USRB'
# #track = sp.track(track_uri)
# #print(track)
# print (sp.audio_analysis(track_uri))

retrying ...2secs
retrying ...1secs
retrying ...1secs


,Name,Date Released,Popularity,Danceability,Energy,Loudness,Speechiness,Acousticness,Liveness,Valence,Tempo,Did They Win?
0,Single Ladies (Put a Ring on It),2008-11-14,71,0.426,0.584,-5.293,0.2960,0.03830,0.1880,0.272,193.437,1
1,Need You Now,2010-01-01,69,0.587,0.622,-5.535,0.0303,0.09270,0.2000,0.231,107.943,1
2,Rolling in the Deep,2011-01-19,77,0.730,0.770,-5.114,0.0298,0.13800,0.0473,0.507,104.948,1
3,We Are Young (feat. Janelle Monáe),2012-02-14,75,0.378,0.638,-5.576,0.0750,0.02000,0.0849,0.735,184.086,1
4,Royals,2013-01-01,79,0.674,0.428,-9.504,0.1220,0.12100,0.1320,0.337,84.878,1
5,Stay With Me,2014-01-01,85,0.418,0.420,-6.444,0.0414,0.58800,0.1100,0.184,84.094,1
6,Thinking out Loud,2014-06-21,84,0.781,0.445,-6.061,0.0295,0.47400,0.1840,0.591,78.998,1
7,Hello,2015-11-20,74,0.470,0.431,-6.129,0.0342,0.32900,0.0854,0.289,157.980,1
8,That's What I Like,2016-11-17,83,0.853,0.560,-4.961,0.0406,0.01300,0.0944,0.860,134.066,1
9,This Is America,2018-05-06,77,0.854,0.463,-6.159,0.1370,0.11700,0.3540,0.549,120.024,1


In [134]:
playlist_albums = sp.user_playlist_tracks('22b7g6udnfkubfplnsla3ww5a', '3q7RrjHTPcG1pAL3j4QcJF', fields='items,uri,name,id,total', market='fr')
playlist_albums

albums = ['2dqn5yOQWdyGwOpOIi9O4x']
name = ['Fearless']
date = []
pop = []
dance = []
energy = []
loud = []
speech = []
acous = []
live = []
valence = []
tempo = []
win = []

for i in playlist_albums['items']:
    name.append(i['track']['album']['name'])
    date.append(i['track']['album']['release_date'])
    pop.append(i['track']['popularity'])
    win.append(1)
    albums.append(i['track']['album']['id'])

del albums[1]
del name[1]

danceavg = 0
energyavg = 0
loudavg = 0
speechavg = 0
acousavg = 0
liveavg = 0
valenceavg = 0
tempoavg = 0
count = 0

for a in albums:
    songs = sp.album_tracks(a)
    for i in songs['items']:
        danceavg = danceavg + sp.audio_features('spotify:track:'+i['id'])[0]['danceability']
        energyavg = energyavg + sp.audio_features('spotify:track:'+i['id'])[0]['energy']
        loudavg = loudavg + sp.audio_features('spotify:track:'+i['id'])[0]['loudness']
        speechavg = speechavg + sp.audio_features('spotify:track:'+i['id'])[0]['speechiness']
        acousavg = acousavg + sp.audio_features('spotify:track:'+i['id'])[0]['acousticness']
        liveavg = liveavg + sp.audio_features('spotify:track:'+i['id'])[0]['liveness']
        valenceavg = valenceavg + sp.audio_features('spotify:track:'+i['id'])[0]['valence']
        tempoavg = tempoavg + sp.audio_features('spotify:track:'+i['id'])[0]['tempo']
        count = count + 1
    dance.append(danceavg/count)
    energy.append(energyavg/count)
    loud.append(loudavg/count)
    speech.append(speechavg/count)
    acous.append(acousavg/count)
    live.append(liveavg/count)
    valence.append(valenceavg/count)
    tempo.append(tempoavg/count)
    danceavg = 0
    energyavg = 0
    loudavg = 0
    speechavg = 0
    acousavg = 0
    liveavg = 0
    valenceavg = 0
    tempoavg = 0
    count = 0
    
playlist_albums = sp.user_playlist_tracks('22b7g6udnfkubfplnsla3ww5a', '6ua8ZJ16YFgCou0ecqPgdV', fields='items,uri,name,id,total', market='fr')
playlist_albums

albums = []

for i in playlist_albums['items']:
    name.append(i['track']['album']['name'])
    date.append(i['track']['album']['release_date'])
    pop.append(i['track']['popularity'])
    win.append(0)
    albums.append(i['track']['album']['id'])

danceavg = 0
energyavg = 0
loudavg = 0
speechavg = 0
acousavg = 0
liveavg = 0
valenceavg = 0
tempoavg = 0
count = 0

    
for a in albums:
    songs = sp.album_tracks(a)
    for i in songs['items']:
        danceavg = danceavg + sp.audio_features('spotify:track:'+i['id'])[0]['danceability']
        energyavg = energyavg + sp.audio_features('spotify:track:'+i['id'])[0]['energy']
        loudavg = loudavg + sp.audio_features('spotify:track:'+i['id'])[0]['loudness']
        speechavg = speechavg + sp.audio_features('spotify:track:'+i['id'])[0]['speechiness']
        acousavg = acousavg + sp.audio_features('spotify:track:'+i['id'])[0]['acousticness']
        liveavg = liveavg + sp.audio_features('spotify:track:'+i['id'])[0]['liveness']
        valenceavg = valenceavg + sp.audio_features('spotify:track:'+i['id'])[0]['valence']
        tempoavg = tempoavg + sp.audio_features('spotify:track:'+i['id'])[0]['tempo']
        count = count + 1
    dance.append(danceavg/count)
    energy.append(energyavg/count)
    loud.append(loudavg/count)
    speech.append(speechavg/count)
    acous.append(acousavg/count)
    live.append(liveavg/count)
    valence.append(valenceavg/count)
    tempo.append(tempoavg/count)
    danceavg = 0
    energyavg = 0
    loudavg = 0
    speechavg = 0
    acousavg = 0
    liveavg = 0
    valenceavg = 0
    tempoavg = 0
    count = 0

data = pd.DataFrame()
data.insert(0, "Name", name, True)
data.insert(1, "Date Released", date, True)
data.insert(2, "Popularity", pop, True)
data.insert(3, "Danceability", dance, True)
data.insert(4, "Energy", energy, True)
data.insert(5, "Loudness", loud, True)
data.insert(6, "Speechiness", speech, True)
data.insert(7, "Acousticness", acous, True)
data.insert(8, "Liveness", live, True)
data.insert(9, "Valence", valence, True)
data.insert(10, "Tempo", tempo, True)
data.insert(11, "Did They Win?", win, True)

data

retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
[60, 70, 77, 52, 64, 47, 71, 74, 83, 72, 79, 69, 77, 18, 31, 69, 79, 39, 73, 75, 5

,Name,Date Released,Popularity,Danceability,Energy,Loudness,Speechiness,Acousticness,Liveness,Valence,Tempo,Did They Win?
0,Fearless,2008-11-10,60,0.592769,0.637308,-5.276000,0.032992,0.175937,0.145554,0.384231,113.235462,1
1,The Suburbs,2010,70,0.448625,0.721381,-7.534563,0.045250,0.217593,0.205238,0.418344,127.591062,1
2,21,2011-01-19,77,0.581091,0.534091,-6.081727,0.036018,0.439371,0.117536,0.414636,133.496909,1
3,Babel,2012-09-21,52,0.404250,0.558167,-8.333750,0.038817,0.185863,0.127025,0.281533,116.001083,1
4,Random Access Memories,2013-05-17,64,0.686462,0.585846,-10.704077,0.044315,0.287906,0.132877,0.488231,114.471077,1
5,Morning Phase,2014-01-01,47,0.463354,0.399923,-9.842538,0.030169,0.430708,0.125154,0.220692,114.185923,1
6,1989,2014-01-01,71,0.650231,0.698769,-6.236462,0.066254,0.113175,0.154577,0.461908,118.552462,1
7,25,2015-11-20,74,0.484636,0.499818,-6.171909,0.043718,0.368573,0.148036,0.294091,130.332000,1
8,24K Magic,2016-11-17,83,0.690556,0.690333,-5.385222,0.066989,0.042790,0.135267,0.695889,136.310111,1
9,Golden Hour,2018-03-30,72,0.613692,0.534385,-7.148846,0.036200,0.412438,0.109492,0.459615,108.531462,1


In [138]:
data.to_csv('topalbums.csv', encoding='utf-8', index=False)
data2 = pd.read_csv("topalbums.csv")

data2


,Name,Date Released,Popularity,Danceability,Energy,Loudness,Speechiness,Acousticness,Liveness,Valence,Tempo,Did They Win?
0,Fearless,2008-11-10,60,0.592769,0.637308,-5.276000,0.032992,0.175937,0.145554,0.384231,113.235462,1
1,The Suburbs,2010,70,0.448625,0.721381,-7.534563,0.045250,0.217593,0.205238,0.418344,127.591062,1
2,21,2011-01-19,77,0.581091,0.534091,-6.081727,0.036018,0.439371,0.117536,0.414636,133.496909,1
3,Babel,2012-09-21,52,0.404250,0.558167,-8.333750,0.038817,0.185863,0.127025,0.281533,116.001083,1
4,Random Access Memories,2013-05-17,64,0.686462,0.585846,-10.704077,0.044315,0.287906,0.132877,0.488231,114.471077,1
5,Morning Phase,2014-01-01,47,0.463354,0.399923,-9.842538,0.030169,0.430708,0.125154,0.220692,114.185923,1
6,1989,2014-01-01,71,0.650231,0.698769,-6.236462,0.066254,0.113175,0.154577,0.461908,118.552462,1
7,25,2015-11-20,74,0.484636,0.499818,-6.171909,0.043718,0.368573,0.148036,0.294091,130.332000,1
8,24K Magic,2016-11-17,83,0.690556,0.690333,-5.385222,0.066989,0.042790,0.135267,0.695889,136.310111,1
9,Golden Hour,2018-03-30,72,0.613692,0.534385,-7.148846,0.036200,0.412438,0.109492,0.459615,108.531462,1
